In [1]:
import dagshub
import mlflow
dagshub.init(url="https://dagshub.com/PacoTinoco/Proyecto_Final_CDD", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="amazon_stock")

Accessing as PacoTinoco

Initialized MLflow to track repo "PacoTinoco/Proyecto_Final_CDD"

Repository PacoTinoco/Proyecto_Final_CDD initialized!

https://dagshub.com/PacoTinoco/Proyecto_Final_CDD.mlflow


<Experiment: artifact_location='mlflow-artifacts:/33bcf767e68f4559aeed57be97a38cdb', creation_time=1727551840815, experiment_id='1', last_update_time=1727551840815, lifecycle_stage='active', name='amazon_stock', tags={}>

In [2]:
import yfinance as yf
from datetime import date, timedelta

# Test dates
amazon_stock = yf.download('AMZN', start='2020-01-01', end='2024-01-01')

[*********************100%***********************]  1 of 1 completed


In [3]:
# Definir X e y
X = amazon_stock.drop(columns=["Close"])
y = amazon_stock["Close"]
from sklearn.model_selection import train_test_split

# Dividir en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("Tamaño del conjunto de entrenamiento:", X_train.shape, y_train.shape)
print("Tamaño del conjunto de prueba:", X_val.shape, y_val.shape)


Tamaño del conjunto de entrenamiento: (804, 5) (804,)
Tamaño del conjunto de prueba: (202, 5) (202,)


In [4]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib
from sklearn.metrics import  root_mean_squared_error
import numpy as np

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [5]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [6]:
from sklearn.metrics import mean_squared_error
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "Amazon Stock validation",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)

        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

[0]	validation-rmse:26.49598                          
[1]	validation-rmse:25.06851                          
[2]	validation-rmse:23.72176                          
[3]	validation-rmse:22.44340                          
[4]	validation-rmse:21.23674                          
[5]	validation-rmse:20.09448                          
[6]	validation-rmse:19.00482                          
[7]	validation-rmse:17.97889                          
[8]	validation-rmse:17.01097                          
[9]	validation-rmse:16.09126                          
[10]	validation-rmse:15.22492                         
[11]	validation-rmse:14.40314                         
[12]	validation-rmse:13.62889                         
[13]	validation-rmse:12.89601                         
[14]	validation-rmse:12.20159                         
[15]	validation-rmse:11.54795                         
[16]	validation-rmse:10.92749                         
[17]	validation-rmse:10.34078                         
[18]	valid

2024/11/03 18:02:21 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/11/03 18:02:21 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\PycharmProjects\Proyecto_Final_CDD\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:02:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/11/03 18:02:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\Francisco\PycharmProjects\Proyecto_Final_CDD\.venv\Lib\site-packages\xgboost\core.py

[0]	validation-rmse:23.54905                                                    
[1]	validation-rmse:19.77660                                                    
[2]	validation-rmse:16.61771                                                    
[3]	validation-rmse:13.99201                                                    
[4]	validation-rmse:11.76317                                                    
[5]	validation-rmse:9.91083                                                     
[6]	validation-rmse:8.34946                                                     
[7]	validation-rmse:7.03206                                                     
[8]	validation-rmse:5.92626                                                     
[9]	validation-rmse:4.99786                                                     
[10]	validation-rmse:4.21554                                                    
[11]	validation-rmse:3.56089                                                    
[12]	validation-rmse:3.01147

2024/11/03 18:03:00 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/11/03 18:03:00 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\PycharmProjects\Proyecto_Final_CDD\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:03:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/11/03 18:03:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\Francisco\PycharmProjects\Proyecto_Final_CDD\.venv\Lib\site-packages\xgboost\core.py

[0]	validation-rmse:23.67100                                                    
[1]	validation-rmse:19.99340                                                    
[2]	validation-rmse:16.87583                                                    
[3]	validation-rmse:14.25157                                                    
[4]	validation-rmse:12.05290                                                    
[5]	validation-rmse:10.17915                                                    
[6]	validation-rmse:8.59940                                                     
[7]	validation-rmse:7.27624                                                     
[8]	validation-rmse:6.15786                                                     
[9]	validation-rmse:5.21112                                                     
[10]	validation-rmse:4.41509                                                    
[11]	validation-rmse:3.74207                                                    
[12]	validation-rmse:3.17646

...

[0]	validation-rmse:4.41695                                                      
[1]	validation-rmse:0.90831                                                      
[2]	validation-rmse:0.51851                                                      
[3]	validation-rmse:0.50595                                                      
[4]	validation-rmse:0.51764                                                      
[5]	validation-rmse:0.52015                                                      
[6]	validation-rmse:0.50170                                                      
[7]	validation-rmse:0.50143                                                      
[8]	validation-rmse:0.50022                                                      
[9]	validation-rmse:0.50103                                                      
[10]	validation-rmse:0.50089                                                     
[11]	validation-rmse:0.49833                                                     
[12]	validation-

2024/11/03 18:07:08 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/11/03 18:07:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\PycharmProjects\Proyecto_Final_CDD\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:07:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/11/03 18:07:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\Francisco\PycharmProjects\Proyecto_Final_CDD\.venv\Lib\site-packages\xgboost\core.py

100%|██████████| 10/10 [05:11<00:00, 31.12s/trial, best loss: 0.33639500880409207]
[0]	validation-rmse:25.14187
[1]	validation-rmse:22.55727
[2]	validation-rmse:20.23117
[3]	validation-rmse:18.16594
[4]	validation-rmse:16.29050
[5]	validation-rmse:14.63450
[6]	validation-rmse:13.14859
[7]	validation-rmse:11.79486
[8]	validation-rmse:10.59607
[9]	validation-rmse:9.52009
[10]	validation-rmse:8.54853
[11]	validation-rmse:7.67454
[12]	validation-rmse:6.89679
[13]	validation-rmse:6.19508
[14]	validation-rmse:5.56971
[15]	validation-rmse:5.00301
[16]	validation-rmse:4.50194
[17]	validation-rmse:4.05190
[18]	validation-rmse:3.64510
[19]	validation-rmse:3.28175
[20]	validation-rmse:2.95762
[21]	validation-rmse:2.66614
[22]	validation-rmse:2.40462
[23]	validation-rmse:2.17071
[24]	validation-rmse:1.95996
[25]	validation-rmse:1.77325
[26]	validation-rmse:1.60538
[27]	validation-rmse:1.45403
[28]	validation-rmse:1.31848
[29]	validation-rmse:1.19945
[30]	validation-rmse:1.09215
[31]	validation-rms

2024/11/03 18:07:33 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/11/03 18:07:33 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\PycharmProjects\Proyecto_Final_CDD\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:07:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/11/03 18:07:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/03 18:07:42 INFO mlflow.tracking._tracking_service.client: 🏃 View run Xgboost Hyper-para

FileNotFoundError: [Errno 2] No such file or directory: 'models/preprocessor.b'

In [10]:
best_params

{'learning_rate': np.float64(0.10350591420643548),
 'max_depth': 9,
 'min_child_weight': np.float64(0.40519944097241933),
 'reg_alpha': np.float64(0.024115199377555675),
 'reg_lambda': np.float64(0.17607909626473248),
 'seed': 42,
 'objective': 'reg:squarederror'}

In [9]:
import mlflow

In [11]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="amazon-stock-model"
)

Successfully registered model 'amazon-stock-model'.
2024/11/03 18:18:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: amazon-stock-model, version 1
Created version '1' of model 'amazon-stock-model'.


In [12]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="amazon-stock-model",
    description="Model registry for the amazon stock prediction",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="amazon-stock-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="amazon-stock-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1730679513749, current_stage='None', description='The model version 1 was transitioned to champion on 2024-11-03 18:19:22.995855', last_updated_timestamp=1730679563450, name='amazon-stock-model', run_id='9236f36cbb31410882121455edc793b3', run_link='', source='mlflow-artifacts:/33bcf767e68f4559aeed57be97a38cdb/9236f36cbb31410882121455edc793b3/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [13]:
import mlflow.pyfunc

model_name = "amazon-stock-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

array([138.18587 , 112.905556, 123.04291 , 161.96548 , 169.20581 ,
       137.67403 , 142.66216 , 158.04428 , 170.06548 , 113.90906 ,
       135.9366  , 174.47244 ,  93.57865 , 166.83345 , 159.21414 ,
       124.71495 , 114.926575, 146.53857 , 153.06241 , 119.70925 ,
       119.79099 , 110.93862 , 161.91837 , 155.19981 , 108.70469 ,
       163.52872 , 171.17198 , 159.02191 , 178.09357 , 127.11529 ,
        90.74072 ,  93.895294, 124.673   , 160.06203 ,  88.4483  ,
       118.338394, 132.1464  , 160.49059 , 156.3703  , 165.46138 ,
       169.94296 , 150.76903 , 170.9801  , 131.95056 , 158.02742 ,
       149.06982 , 185.11769 , 134.94225 , 115.22805 , 129.2039  ,
       108.482544,  90.088806,  94.98959 , 113.05485 , 161.63956 ,
        97.40563 , 185.6042  ,  92.36454 , 101.304085, 106.890434,
       129.09166 , 173.52603 ,  93.996574, 103.10434 ,  98.53486 ,
       175.849   , 165.03674 , 166.85509 , 169.51318 , 102.26561 ,
       124.045746, 156.78093 , 173.95172 , 158.5232  ,  88.230